In [1]:
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import numpy as np
from scipy.optimize import minimize
import pandas as pd
from scipy.stats import poisson, uniform, expon, pareto
from tqdm import tqdm
from mdptoolbox import mdp, util
import itertools
from scipy.sparse import csr_matrix, lil_matrix
from matplotlib.patches import Patch
import math
import random
import sympy as sp
from sympy.printing.latex import print_latex

# G calcs

In [21]:
# linear attack

In [2]:
p0, p0p, p1, p0pp = sp.symbols('p_0 p_{00} p_1 p_{000}')
a, b, x, g, p, C, E  = sp.symbols('a b x g p C E')

In [3]:
# ps should be the same
p0    = p1 / (a*(1-p)*(1-sp.exp(-b+C)) + a*p*(1-sp.Min(sp.exp(-b+C+E),1)))
p0p   = p1 * (1-a)
p0pp  = p1 * a
p2    = p1*(a/(1-a))
prest = p1*a/(1-2*a)

In [4]:
eq = sp.Eq(p0+p0p+p0pp+prest+p1, 1)
p1_sym = sp.solve(eq, p1)[0]

In [5]:
p1_me = (1/(a*(1-p)*(1-sp.exp(-b+C)) + a*p*(1-sp.Min(sp.exp(-b+C+E),1))) + 1 + (1-a)/(1-2*a))**(-1)
p1_me

1/(1 + 1/(a*p*(1 - Min(1, exp(C + E - b))) + a*(1 - p)*(1 - exp(C - b))) + (1 - a)/(1 - 2*a))

In [6]:
p1_sym.evalf(subs={a:0.3, b:3, p:0.5, C:1, E:1}), p1_me.evalf(subs={a:0.3, b:3, p:0.5, C:1, E:1})

(0.138811963706275, 0.138811963706275)

In [15]:
f0i   = a*p*(sp.Max(0,b-C-E)+1)*sp.Min(1,sp.exp(-b+C+E)) + a*(1-p)*(b-C+1)*sp.exp(-b+C)
f0ii  = a* (a*p*(1-(sp.Max(0,b-C-E)+1)*sp.Min(1,sp.exp(-b+C+E))) + a*(1-p)*(1-(b-C+1)*sp.exp(-b+C))) 
f0iii = ((1-a)*(a+g*(1-a)))* (a*p*(1-(sp.Max(0,b-C-E)+1)*sp.Min(1,sp.exp(-b+C+E))) + a*(1-p)*(1-(b-C+1)*sp.exp(-b+C)))
f0iii

(1 - a)*(a + g*(1 - a))*(a*p*(-(Max(0, -C - E + b) + 1)*Min(1, exp(C + E - b)) + 1) + a*(1 - p)*(-(-C + b + 1)*exp(C - b) + 1))

In [16]:
f0iii.evalf(subs={p:0, C:0.})

a*(1.0 - a)*(a + g*(1.0 - a))*(-(b + 1.0)*exp(-b) + 1.0)

In [17]:
f0 = f0i + f0ii + f0iii
f1 = a + 2*a*(1-a)

In [18]:
full = p0*f0 + p1*f1 + p1*a*a*(3-2*a)/(1-2*a)
p1_ev = p1_sym.evalf(subs={C:1., E:1.})
final = sp.simplify(full.evalf(subs={p1:p1_ev, C:1., E:1., g:0}))
final

0.122626480390481*a*(a**2*(2.0*a - 3.0)*(p*(Min(1.0, 7.38905609893065*exp(-b)) - 1.0)*exp(b) + (p - 1.0)*(1.0*exp(b) - 2.71828182845905)) - a*(2.0*a - 3.0)*(2.0*a - 1.0)*(p*(Min(1.0, 7.38905609893065*exp(-b)) - 1.0)*exp(b) + (p - 1.0)*(1.0*exp(b) - 2.71828182845905)) + (2.0*a - 1.0)*(-a*(a - 1.0)*(p*((Max(0, b - 2.0) + 1.0)*Min(1.0, 7.38905609893065*exp(-b)) - 1.0)*exp(b) - (2.71828182845905*b - 1.0*exp(b))*(p - 1.0)) + a*(p*((Max(0, b - 2.0) + 1.0)*Min(1.0, 7.38905609893065*exp(-b)) - 1.0)*exp(b) - (2.71828182845905*b - 1.0*exp(b))*(p - 1.0)) + 2.71828182845905*b*(p - 1.0) - p*(Max(0, b - 2.0) + 1.0)*exp(b)*Min(1.0, 7.38905609893065*exp(-b))))*(-5.43656365691809*a*p + 5.43656365691809*a + 2.71828182845905*p + (2.0*a*p*Min(1.0, 7.38905609893065*exp(-b)) - 2.0*a - p*Min(1.0, 7.38905609893065*exp(-b)) + 1.0)*exp(b) - 2.71828182845905)/((2.0*a - 1.0)*(p*(Min(1.0, 7.38905609893065*exp(-b)) - 1.0)*exp(b) + (p - 1.0)*(1.0*exp(b) - 2.71828182845905))*(-a**2*p + a**2 + 0.666666666666667*a*p - 

In [19]:
finalLambda = sp.lambdify((a, b, p), final)
finalLambda

<function _lambdifygenerated(a, b, p)>

In [14]:
a_ = 0.3
b_ = 2.2
p_ = 0.5
finalLambda(a_, b_, p_)

0.32299291331383856

In [20]:
# linear honest 

In [22]:
p0, p0p, p1, p0pp = sp.symbols('p_0 p_{00} p_1 p_{000}')
a, b, x, g, p, C, E  = sp.symbols('a b x g p C E')

In [23]:
# ps should be the same
p0    = p1 / (a*(1-p)*(1-sp.exp(-b+C)) + a*p*(1-sp.Min(sp.exp(-b+C+E),1)))
p0p   = p1 * (1-a)
p0pp  = p1 * a
p2    = p1*(a/(1-a))
prest = p1*a/(1-2*a)

In [24]:
eq = sp.Eq(p0+p0p+p0pp+prest+p1, 1)
p1_sym = sp.solve(eq, p1)[0]

In [25]:
p1_me = (1/(a*(1-p)*(1-sp.exp(-b+C)) + a*p*(1-sp.Min(sp.exp(-b+C+E),1))) + 1 + (1-a)/(1-2*a))**(-1)
p1_me

1/(1 + 1/(a*p*(1 - Min(1, exp(C + E - b))) + a*(1 - p)*(1 - exp(C - b))) + (1 - a)/(1 - 2*a))

In [26]:
p1_sym.evalf(subs={a:0.3, b:3, p:0.5, C:1, E:1}), p1_me.evalf(subs={a:0.3, b:3, p:0.5, C:1, E:1})

(0.138811963706275, 0.138811963706275)

In [92]:
g0  = (1-a) + (1-a)*(1-a)*(1-g)*(a*p*(1-(sp.Max(0,b-C-E)+1)*sp.Min(1,sp.exp(-b+C+E))) + a*(1-p)*(1-(b-C+1)*sp.exp(-b+C)))
g0p = (1-a)

In [89]:
gull = p0*g0 + p0p*g0p + p1*a*(2-a)
p1_ev = p1_sym.evalf(subs={C:1., E:1.})
ginal = gull.evalf(subs={p1:p1_ev, C:1., E:1., g:0})
ginal

a**2*(2.0 - a)*(-2.0*a*p*Min(1.0, 7.38905609893065*exp(-b)) + 5.43656365691809*a*p*exp(-b) + 2.0*a - 5.43656365691809*a*exp(-b) + p*Min(1.0, 7.38905609893065*exp(-b)) - 2.71828182845905*p*exp(-b) - 1.0 + 2.71828182845905*exp(-b))/(-3.0*a**2*p*Min(1.0, 7.38905609893065*exp(-b)) + 8.15484548537714*a**2*p*exp(-b) + 3.0*a**2 - 8.15484548537714*a**2*exp(-b) + 2.0*a*p*Min(1.0, 7.38905609893065*exp(-b)) - 5.43656365691809*a*p*exp(-b) + 5.43656365691809*a*exp(-b) - 1.0) + a*(1.0 - a)**2*(-2.0*a*p*Min(1.0, 7.38905609893065*exp(-b)) + 5.43656365691809*a*p*exp(-b) + 2.0*a - 5.43656365691809*a*exp(-b) + p*Min(1.0, 7.38905609893065*exp(-b)) - 2.71828182845905*p*exp(-b) - 1.0 + 2.71828182845905*exp(-b))/(-3.0*a**2*p*Min(1.0, 7.38905609893065*exp(-b)) + 8.15484548537714*a**2*p*exp(-b) + 3.0*a**2 - 8.15484548537714*a**2*exp(-b) + 2.0*a*p*Min(1.0, 7.38905609893065*exp(-b)) - 5.43656365691809*a*p*exp(-b) + 5.43656365691809*a*exp(-b) - 1.0) + a*(-a + (1.0 - a)**2*(p*(-(Max(0, b - 2.0) + 1.0)*Min(1.0, 7.3

In [90]:
ginalLambda = sp.lambdify((a, b, p), ginal)
ginalLambda

<function _lambdifygenerated(a, b, p)>

In [91]:
a_ = 0.3
b_ = 2.2
p_ = 0.5
finalLambda(a_, b_, p_), ginalLambda(a_, b_, p_), finalLambda(a_, b_, p_) + ginalLambda(a_, b_, p_)

(0.32299291331383856, 0.674203863321568, 0.9971967766354066)